# Basic feature engineering
- Need to figure out basic feature pipelines for previous season and for current season


In [1]:
import duckdb

In [3]:
import pandas

In [2]:
db = "../Data/fb.db"

In [4]:
conn = duckdb.connect(db)

In [8]:
conn.sql("select Season, Div, Date, FTR from ResultsRaw limit 100").to_df()

,Season,Div,Date,FTR
0,1415,E1,2014-08-08,D
1,1415,E1,2014-08-09,D
2,1415,E1,2014-08-09,A
3,1415,E1,2014-08-09,H
4,1415,E1,2014-08-09,A
...,...,...,...,...
95,1415,E1,2014-09-22,A
96,1415,E1,2014-09-27,A
97,1415,E1,2014-09-27,D
98,1415,E1,2014-09-27,A


## Team Results table
- First going to create a table with result/stats for a team for a game on a single line rather than having stats for two teams on one line

In [200]:

homeCols = ['Div', 'Season', 'Date', 'HomeTeam', 'FTR', 'FTHG', 'FTAG', 'HS', '"AS"',
                   'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR']
"""
homeCols = {'HomeTeam': 'Team',
            'FTHG': 'Goals_Fr',
            'FTAG': 'Goals_Ag',
            'HS': 'Shots_Fr',
            '"AS"': 'Shots_Ag', 
            'HST': 'ShotsT_Fr', 
            'AST': 'ShotsT_Ag',
            'HF': 'Fouls_Fr',
            'AF': 'Fouls_Ag',
            'HC': 'Corners_Fr', 
            'AC': 'Corners_Ag',
            'HY': 'Yc_Fr',
            'AY': 'Yc_Ag', 
            'HR': 'Rc_Fr',
            'AR': 'Rc_Ag'
           }

"""


awayCols = ['Div', 'Season', 'Date', 'AwayTeam','FTR', 'FTHG', 'FTAG', 'HS', '"AS"',
            'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR']
"""
awayCols = {'AwayTeam': 'Team',
            'FTHG': 'Goals_Ag',
            'FTAG': 'Goals_Fr',
            'HS': 'Shots_Ag',
            '"AS"': 'Shots_Fr', 
            'HST': 'ShotsT_Ag', 
            'AST': 'ShotsT_Fr',
            'HF': 'Fouls_Ag',
            'AF': 'Fouls_Fr',
            'HC': 'Corners_Ag', 
            'AC': 'Corners_Fr',
            'HY': 'Yc_Ag',
            'AY': 'Yc_Fr', 
            'HR': 'Rc_Ag',
            'AR': 'Rc_Fr'
           }
"""

'\nawayCols = {\'AwayTeam\': \'Team\',\n            \'FTHG\': \'Goals_Ag\',\n            \'FTAG\': \'Goals_Fr\',\n            \'HS\': \'Shots_Ag\',\n            \'"AS"\': \'Shots_Fr\', \n            \'HST\': \'ShotsT_Ag\', \n            \'AST\': \'ShotsT_Fr\',\n            \'HF\': \'Fouls_Ag\',\n            \'AF\': \'Fouls_Fr\',\n            \'HC\': \'Corners_Ag\', \n            \'AC\': \'Corners_Fr\',\n            \'HY\': \'Yc_Ag\',\n            \'AY\': \'Yc_Fr\', \n            \'HR\': \'Rc_Ag\',\n            \'AR\': \'Rc_Fr\'\n           }\n'

In [177]:
homeQueryBuilder = []
for col in homeCols:
    if "H" in col:
        if col == "HomeTeam":
            mapping = "Team"

        elif col[:2] == "FT":
            mapping = "G_F"

        else:
            mapping = "{}_F".format(col[1:])

    elif "A" in col:
        if col[:2] == "FT":
            mapping = "G_Ag"

        elif col == '"AS"':
            mapping = "S_Ag"

        else:
            mapping = "{}_Ag".format(col[1:])

    else:
        mapping = col

    if "FTR" in col:
        homeQueryBuilder.append("case when \"FTR\" = 'H' then 3 when \"FTR\" = 'D' then 1 else 0 end as Pts")
    else:
        homeQueryBuilder.append("{} as {}".format(col, mapping))

In [178]:
homeQuery = "Select {}, TRUE as AtHome from ResultsRaw".format(", ".join(homeQueryBuilder))

In [179]:
print_query(homeQuery)

Select Div as Div,
 Season as Season,
 Date as Date,
 HomeTeam as Team,
 case when "FTR" = 'H' then 3 when "FTR" = 'D' then 1 else 0 end as Pts,
 FTHG as G_F,
 FTAG as G_Ag,
 HS as S_F,
 "AS" as S_Ag,
 HST as ST_F,
 AST as ST_Ag,
 HF as F_F,
 AF as F_Ag,
 HC as C_F,
 AC as C_Ag,
 HY as Y_F,
 AY as Y_Ag,
 HR as R_F,
 AR as R_Ag,
 TRUE as AtHome,
from ResultsRa
w


In [180]:
conn.sql(homeQuery).to_df().head()

,Div,Season,Date,Team,Pts,G_F,G_Ag,S_F,S_Ag,ST_F,ST_Ag,F_F,F_Ag,C_F,C_Ag,Y_F,Y_Ag,R_F,R_Ag,AtHome
0,E1,1415,2014-08-08,Blackburn,1,1,1,12,6,3,3,6,14,4,5,1,3,0,0.0,True
1,E1,1415,2014-08-09,Brentford,1,1,1,14,13,4,4,3,13,11,5,1,1,0,0.0,True
2,E1,1415,2014-08-09,Brighton,0,0,1,11,5,3,1,13,13,4,4,3,1,1,0.0,True
3,E1,1415,2014-08-09,Derby,3,1,0,10,16,3,6,8,11,8,3,0,1,0,0.0,True
4,E1,1415,2014-08-09,Huddersfield,0,0,4,16,12,5,7,15,8,7,4,2,1,0,0.0,True


In [191]:
homeRemap = conn.sql(homeQuery)

In [182]:
conn.sql('Select Season, Team, sum(G_F) as G_F_tot from homeRemap where Team = \'Birmingham\' group by Season, Team').to_df()

,Season,Team,G_F_tot
0,1415,Birmingham,29.0
1,1314,Birmingham,29.0
2,1718,Birmingham,21.0
3,1617,Birmingham,25.0
4,1011,Birmingham,19.0
5,1516,Birmingham,27.0
6,1213,Birmingham,29.0
7,1112,Birmingham,37.0


In [201]:
awayQueryBuilder = []
for col in awayCols:
    if "A" in col:
        if col == "AwayTeam":
            mapping = "Team"

        elif col[:2] == "FT":
            mapping = "G_F"

        elif col == '"AS"':
            mapping = "S_F"

        else:
            mapping = "{}_F".format(col[1:])

    elif "H" in col:
        if col[:2] == "FT":
            mapping = "G_Ag"

        else:
            mapping = "{}_Ag".format(col[1:])

    else:
        mapping = col

    if "FTR" in col:
        awayQueryBuilder.append("case when \"FTR\" = 'A' then 3 when \"FTR\" = 'D' then 1 else 0 end as Pts")
    else:
        awayQueryBuilder.append("{} as {}".format(col, mapping))

In [202]:
awayQuery = "select {}, FALSE as AtHome from ResultsRaw".format(", ".join(awayQueryBuilder))

In [203]:
awayRemap = conn.sql(awayQuery)

In [204]:
awayRemap.to_df().head()

,Div,Season,Date,Team,Pts,G_Ag,G_F,S_Ag,S_F,ST_Ag,ST_F,F_Ag,F_F,C_Ag,C_F,Y_Ag,Y_F,R_Ag,R_F,AtHome
0,E1,1415,2014-08-08,Cardiff,1,1,1,12,6,3,3,6,14,4,5,1,3,0,0.0,False
1,E1,1415,2014-08-09,Charlton,1,1,1,14,13,4,4,3,13,11,5,1,1,0,0.0,False
2,E1,1415,2014-08-09,Sheffield Weds,3,0,1,11,5,3,1,13,13,4,4,3,1,1,0.0,False
3,E1,1415,2014-08-09,Rotherham,0,1,0,10,16,3,6,8,11,8,3,0,1,0,0.0,False
4,E1,1415,2014-08-09,Bournemouth,3,0,4,16,12,5,7,15,8,7,4,2,1,0,0.0,False


In [205]:
conn.sql('Select Season, Team, sum(G_F) as G_F_tot from awayRemap where Team = \'Blackpool\' group by Season, Team').to_df()

,Season,Team,G_F_tot
0,1213,Blackpool,30.0
1,1112,Blackpool,37.0
2,1415,Blackpool,18.0
3,1718,Blackpool,23.0
4,1314,Blackpool,18.0
5,1011,Blackpool,25.0
6,1516,Blackpool,18.0


In [206]:
cols = list(homeRemap.to_df().columns)

In [207]:
conn.sql("""create or replace table TeamResults as 
            select {} 
            from homeRemap 
            union 
            select  {} 
            from awayRemap
            """.format(",".join(cols), ",".join(cols)))

In [208]:
conn.sql("select * from TeamResults").to_df().head()

,Div,Season,Date,Team,Pts,G_F,G_Ag,S_F,S_Ag,ST_F,ST_Ag,F_F,F_Ag,C_F,C_Ag,Y_F,Y_Ag,R_F,R_Ag,AtHome
0,E1,1415,2014-08-09,Nott'm Forest,3,2,0,15,7,4,1,14,10,7,2,1,2,0.0,0.0,True
1,E1,1415,2014-08-16,Cardiff,3,3,1,16,21,9,7,13,13,6,9,0,3,0.0,0.0,True
2,E1,1415,2014-08-16,Fulham,0,0,1,24,11,7,6,16,15,11,2,3,0,0.0,0.0,True
3,E1,1415,2014-08-16,Reading,3,1,0,6,15,1,3,15,14,4,6,2,3,0.0,0.0,True
4,E1,1415,2014-08-19,Reading,0,1,2,16,10,2,4,9,8,5,2,2,2,0.0,0.0,True


Sets up a table with the result for each team, with a single line for each team.
Will make further work easier in this format

## Previous season

In [212]:
statCols = [col for col in cols if col not in ("Div","Season", "Date", "Team", "AtHome")]

In [213]:
statQueryBuilder = ["sum({}) as {}_S_Tot".format(col, col) for col in statCols]

In [214]:
statQueryBuilder

['sum(Pts) as Pts_S_Tot',
 'sum(G_F) as G_F_S_Tot',
 'sum(G_Ag) as G_Ag_S_Tot',
 'sum(S_F) as S_F_S_Tot',
 'sum(S_Ag) as S_Ag_S_Tot',
 'sum(ST_F) as ST_F_S_Tot',
 'sum(ST_Ag) as ST_Ag_S_Tot',
 'sum(F_F) as F_F_S_Tot',
 'sum(F_Ag) as F_Ag_S_Tot',
 'sum(C_F) as C_F_S_Tot',
 'sum(C_Ag) as C_Ag_S_Tot',
 'sum(Y_F) as Y_F_S_Tot',
 'sum(Y_Ag) as Y_Ag_S_Tot',
 'sum(R_F) as R_F_S_Tot',
 'sum(R_Ag) as R_Ag_S_Tot']

In [217]:
statsQuery = "select Div, Season, Team, {} from TeamResults group by Div, Season, Team".format(", ".join(statQueryBuilder))

In [218]:
statsQuery

'select Div, Season, Team, sum(Pts) as Pts_S_Tot, sum(G_F) as G_F_S_Tot, sum(G_Ag) as G_Ag_S_Tot, sum(S_F) as S_F_S_Tot, sum(S_Ag) as S_Ag_S_Tot, sum(ST_F) as ST_F_S_Tot, sum(ST_Ag) as ST_Ag_S_Tot, sum(F_F) as F_F_S_Tot, sum(F_Ag) as F_Ag_S_Tot, sum(C_F) as C_F_S_Tot, sum(C_Ag) as C_Ag_S_Tot, sum(Y_F) as Y_F_S_Tot, sum(Y_Ag) as Y_Ag_S_Tot, sum(R_F) as R_F_S_Tot, sum(R_Ag) as R_Ag_S_Tot from TeamResults group by Div, Season, Team'

### Print query utility function

In [134]:
def print_query(query):
    select_idx = query.lower().find('select')
    from_idx = query.lower().find('from')
    where_idx = query.lower().find('where')
    group_idx = query.lower().find('group')

    selects = query[select_idx:from_idx].split(",")
    for select in selects:
        print("{},".format(select.rstrip()))

    if where_idx > -1:
        print(query[from_idx:where_idx])
    else:
        print(query[from_idx:group_idx])
    print(query[group_idx:])

In [219]:
print_query(statsQuery)

select Div,
 Season,
 Team,
 sum(Pts) as Pts_S_Tot,
 sum(G_F) as G_F_S_Tot,
 sum(G_Ag) as G_Ag_S_Tot,
 sum(S_F) as S_F_S_Tot,
 sum(S_Ag) as S_Ag_S_Tot,
 sum(ST_F) as ST_F_S_Tot,
 sum(ST_Ag) as ST_Ag_S_Tot,
 sum(F_F) as F_F_S_Tot,
 sum(F_Ag) as F_Ag_S_Tot,
 sum(C_F) as C_F_S_Tot,
 sum(C_Ag) as C_Ag_S_Tot,
 sum(Y_F) as Y_F_S_Tot,
 sum(Y_Ag) as Y_Ag_S_Tot,
 sum(R_F) as R_F_S_Tot,
 sum(R_Ag) as R_Ag_S_Tot,
from TeamResults 
group by Div, Season, Team


In [221]:
conn.sql("create or replace table SeasonSummary as {}".format(statsQuery))

In [222]:
conn.sql("select * from SeasonSummary where Div = 'E1' and Season = '1415'").to_df()

,Div,Season,Team,Pts_S_Tot,G_F_S_Tot,G_Ag_S_Tot,S_F_S_Tot,S_Ag_S_Tot,ST_F_S_Tot,ST_Ag_S_Tot,F_F_S_Tot,F_Ag_S_Tot,C_F_S_Tot,C_Ag_S_Tot,Y_F_S_Tot,Y_Ag_S_Tot,R_F_S_Tot,R_Ag_S_Tot
0,E1,1415,Sheffield Weds,60.0,43.0,49.0,600.0,568.0,166.0,181.0,560.0,518.0,228.0,236.0,86.0,74.0,3.0,4.0
1,E1,1415,Bolton,51.0,54.0,67.0,584.0,728.0,194.0,243.0,571.0,469.0,278.0,269.0,77.0,63.0,2.0,3.0
2,E1,1415,Birmingham,63.0,54.0,64.0,614.0,724.0,204.0,235.0,462.0,554.0,240.0,266.0,53.0,73.0,6.0,3.0
3,E1,1415,Rotherham,49.0,46.0,67.0,618.0,618.0,189.0,203.0,501.0,464.0,227.0,259.0,73.0,53.0,2.0,1.0
4,E1,1415,Charlton,60.0,54.0,60.0,475.0,751.0,168.0,220.0,458.0,401.0,193.0,330.0,59.0,57.0,2.0,3.0
5,E1,1415,Norwich,86.0,88.0,48.0,740.0,431.0,243.0,154.0,545.0,467.0,297.0,203.0,85.0,72.0,4.0,3.0
6,E1,1415,Millwall,41.0,42.0,76.0,610.0,601.0,171.0,217.0,645.0,604.0,235.0,244.0,89.0,67.0,5.0,4.0
7,E1,1415,Brentford,78.0,78.0,59.0,710.0,632.0,266.0,215.0,409.0,580.0,298.0,269.0,73.0,92.0,2.0,8.0
8,E1,1415,Nott'm Forest,59.0,71.0,69.0,673.0,649.0,213.0,205.0,537.0,527.0,248.0,287.0,66.0,74.0,3.0,4.0
9,E1,1415,Watford,89.0,91.0,50.0,668.0,632.0,252.0,206.0,487.0,460.0,252.0,256.0,74.0,82.0,7.0,6.0


## Current Season

### Recent

- Get 5 previous game average for stats
- Will likely expand later but this works for now

In [241]:
statCols

['Pts',
 'G_F',
 'G_Ag',
 'S_F',
 'S_Ag',
 'ST_F',
 'ST_Ag',
 'F_F',
 'F_Ag',
 'C_F',
 'C_Ag',
 'Y_F',
 'Y_Ag',
 'R_F',
 'R_Ag']

In [251]:
aggQueryBuilder = []

for col in statCols:
    temp_query = "AVG({}) OVER (PARTITION BY Div, Season, Team ORDER BY Date ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING) as {}_P5_Avg".format(col, col)
    aggQueryBuilder.append(temp_query)

aggQueryPart = ",".join(aggQueryBuilder)

aggQuery =  """
            CREATE OR REPLACE TABLE TeamStatsRecent as
            SELECT
            Div,
            Season,
            Team,
            Date,
            {}
            FROM TeamResults
            WHERE Div IS NOT NULL
            ORDER BY Div, Season, Team, Date
            
            """.format(aggQueryPart)
            
                

In [252]:
conn.sql(aggQuery)

In [254]:
conn.sql("SELECT * FROM TeamStatsRecent").to_df()

,Div,Season,Team,Date,Pts_P5_Avg,G_F_P5_Avg,G_Ag_P5_Avg,S_F_P5_Avg,S_Ag_P5_Avg,ST_F_P5_Avg,ST_Ag_P5_Avg,F_F_P5_Avg,F_Ag_P5_Avg,C_F_P5_Avg,C_Ag_P5_Avg,Y_F_P5_Avg,Y_Ag_P5_Avg,R_F_P5_Avg,R_Ag_P5_Avg
0,E0,1011,Arsenal,2010-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,1011,Arsenal,2010-08-21,1.000000,1.00,1.000000,14.00,7.000000,7.000000,4.0,15.000000,13.000000,11.0,9.00,3.00,1.000000,1.000000,1.000000
2,E0,1011,Arsenal,2010-08-28,2.000000,3.50,0.500000,20.00,5.000000,11.500000,2.5,12.000000,8.000000,9.5,5.50,1.50,0.500000,0.500000,1.000000
3,E0,1011,Arsenal,2010-09-11,2.333333,3.00,0.666667,18.00,8.333333,10.666667,4.0,10.333333,8.333333,9.0,5.00,1.00,0.666667,0.333333,0.666667
4,E0,1011,Arsenal,2010-09-18,2.500000,3.25,0.750000,19.25,8.000000,11.500000,4.5,10.500000,9.000000,9.0,5.25,1.25,1.000000,0.250000,0.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23739,E2,1718,Wigan,2018-04-17,2.000000,2.60,0.800000,12.80,8.400000,5.600000,2.6,9.600000,12.800000,5.0,3.20,1.20,2.600000,0.000000,0.000000
23740,E2,1718,Wigan,2018-04-21,2.000000,2.20,0.800000,12.20,7.600000,4.400000,2.0,9.600000,12.800000,5.2,3.40,1.20,2.600000,0.000000,0.000000
23741,E2,1718,Wigan,2018-04-24,2.600000,2.80,0.400000,13.00,8.400000,5.200000,1.8,8.600000,11.800000,6.0,3.80,1.00,2.200000,0.000000,0.000000
23742,E2,1718,Wigan,2018-04-28,2.200000,2.00,0.400000,11.40,8.200000,4.400000,1.6,9.800000,11.400000,6.4,3.40,0.60,1.600000,0.000000,0.000000


### Overall

- Get overall season to date averages

In [261]:
aggQueryBuilder = []

for col in statCols:
    temp_query = "AVG({}) OVER (PARTITION BY Div, Season, Team ORDER BY Date ROWS UNBOUNDED PRECEDING EXCLUDE CURRENT ROW) as {}_CS_Avg".format(col, col)
    aggQueryBuilder.append(temp_query)

aggQueryPart = ",".join(aggQueryBuilder)

aggQuery =  """
            CREATE OR REPLACE TABLE TeamStats as
            SELECT
            Div,
            Season,
            Team,
            Date,
            {}
            FROM TeamResults
            WHERE Div IS NOT NULL
            ORDER BY Div, Season, Team, Date
            
            """.format(aggQueryPart)
            
                

In [262]:
conn.sql(aggQuery)

In [265]:
conn.sql("select * from TeamStats").to_df()

,Div,Season,Team,Date,Pts_CS_Avg,G_F_CS_Avg,G_Ag_CS_Avg,S_F_CS_Avg,S_Ag_CS_Avg,ST_F_CS_Avg,ST_Ag_CS_Avg,F_F_CS_Avg,F_Ag_CS_Avg,C_F_CS_Avg,C_Ag_CS_Avg,Y_F_CS_Avg,Y_Ag_CS_Avg,R_F_CS_Avg,R_Ag_CS_Avg
0,E0,1011,Arsenal,2010-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,1011,Arsenal,2010-08-21,1.000000,1.000000,1.000000,14.000000,7.000000,7.000000,4.000000,15.000000,13.000000,11.000000,9.000000,3.000000,1.000000,1.000000,1.000000
2,E0,1011,Arsenal,2010-08-28,2.000000,3.500000,0.500000,20.000000,5.000000,11.500000,2.500000,12.000000,8.000000,9.500000,5.500000,1.500000,0.500000,0.500000,1.000000
3,E0,1011,Arsenal,2010-09-11,2.333333,3.000000,0.666667,18.000000,8.333333,10.666667,4.000000,10.333333,8.333333,9.000000,5.000000,1.000000,0.666667,0.333333,0.666667
4,E0,1011,Arsenal,2010-09-18,2.500000,3.250000,0.750000,19.250000,8.000000,11.500000,4.500000,10.500000,9.000000,9.000000,5.250000,1.250000,1.000000,0.250000,0.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23739,E2,1718,Wigan,2018-04-17,2.121951,1.975610,0.658537,13.658537,7.707317,5.487805,2.365854,11.121951,12.658537,6.268293,3.926829,1.585366,2.048780,0.073171,0.146341
23740,E2,1718,Wigan,2018-04-21,2.142857,1.952381,0.642857,13.619048,7.619048,5.428571,2.333333,11.023810,12.666667,6.285714,3.857143,1.547619,2.071429,0.071429,0.142857
23741,E2,1718,Wigan,2018-04-24,2.162791,2.000000,0.627907,13.627907,7.697674,5.488372,2.348837,10.906977,12.627907,6.395349,3.860465,1.534884,2.046512,0.069767,0.139535
23742,E2,1718,Wigan,2018-04-28,2.136364,1.977273,0.636364,13.477273,7.659091,5.386364,2.318182,11.022727,12.636364,6.386364,3.795455,1.500000,2.000000,0.068182,0.136364


In [266]:
conn.close()